# Generate AI-ready Data

`Input:` level 2 cleaned and clipped data

`Output:` level 3 ai-ready data

This notebook processes the cleaned images that are clipped to the target alluvial fan in Eberswalde crater. This mainly means matching the resolution of the images and changing the underlying shape of the value distributions (i.e., the histograms). This puts the data in a format that is conducive for deep learning algorithms.

You can skip this notebook and just download the level 3 data in the `download_data` notebook if you want. This notebook is here for reproducibility and open-source purposes.

In [33]:
import rioxarray as rxr
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
import os
import gdown
import zipfile
import shutil

data_path = '/Users/tthomas/Desktop/ESS569/MLGEO2024_MarsFans/data_lvl2_cleaned-clipped/' # Where is the level 2 data stored? Change this for your machine
save_path = '/Users/tthomas/Desktop/ESS569/MLGEO2024_MarsFans/data_lvl3_ai-ready/' # Where should the level 3 data be saved? Change this for your machine

In [6]:
def normalize_image(image):
    """
    Normalize the values of an image to be between 0 and 1.

    Parameters:
    image (xarray.DataArray): The image to normalize.

    Returns:
    xarray.DataArray: The normalized image.
    """
    # Get the minimum and maximum values of the image
    min_val = image.min().item()
    max_val = image.max().item()

    # Normalize the image
    normalized_image = (image - min_val) / (max_val - min_val)

    return normalized_image

In [8]:
def plot_image_and_histogram(image, title):
    """
    Plots an image and its histogram side by side.

    Parameters:
    image (xarray.DataArray): The image to plot.
    title (str): The title for the plots.
    """
    # Remove NaN values for the histogram
    data_values = image.values.flatten()
    data_values = data_values[~np.isnan(data_values)]

    # Create the histogram
    n, bins = np.histogram(data_values, bins=50)

    # Create a figure with two subplots
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Plot the histogram
    ax[1].stairs(n, bins)
    ax[1].set_title(f"{title} Histogram")
    ax[1].set_xlabel("Value")
    ax[1].set_ylabel("Frequency")

    # Plot the image
    image.plot(ax=ax[0])
    ax[0].set_title(title)

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [22]:
blendDEM = rxr.open_rasterio(data_path + "blendDEM_clipped.tif", cache=False)
blendSLOPE = rxr.open_rasterio(data_path + "blendSLOPE_clipped.tif", cache=False)

ctxDEM = rxr.open_rasterio(data_path + "ctxDEM_clipped.tif", cache=False)
ctxIMG = rxr.open_rasterio(data_path + "ctxIMG_clipped.tif", cache=False)
ctxSLOPE = rxr.open_rasterio(data_path + "ctxSLOPE_clipped.tif", cache=False)

dayIR = rxr.open_rasterio(data_path + "dayIR_clipped.tif", cache=False)
nightIR = rxr.open_rasterio(data_path + "nightIR_clipped.tif", cache=False)

hrscND = rxr.open_rasterio(data_path + "hrscND_clipped.tif", cache=False)
hrscP1 = rxr.open_rasterio(data_path + "hrscP1_clipped.tif", cache=False)
hrscP2 = rxr.open_rasterio(data_path + "hrscP2_clipped.tif", cache=False)
hrscS1 = rxr.open_rasterio(data_path + "hrscS1_clipped.tif", cache=False)
hrscS2 = rxr.open_rasterio(data_path + "hrscS2_clipped.tif", cache=False)

clipped_imgs = [blendDEM, blendSLOPE, ctxDEM, ctxIMG, ctxSLOPE, dayIR, nightIR, hrscND, hrscP1, hrscP2, hrscS1, hrscS2]
clipped_names = ['blendDEM', 'blendSLOPE', 'ctxDEM', 'ctxIMG', 'ctxSLOPE', 'dayIR', 'nightIR', 'hrscND', 'hrscP1', 'hrscP2', 'hrscS1', 'hrscS2']

# # Plot each image and its histogram
# for img, name in zip(clipped_imgs, clipped_names):
#     plot_image_and_histogram(img, name)

### Processing and normalization

- Remove all values greater than 10 in the CTX slope map
- Take the log of the DEMs
- Normalize to 0-1 range

In [38]:
ctxSLOPE_trim = ctxSLOPE.where(ctxSLOPE <= 10, other=ctxSLOPE.median())

# Shift the DEMs so they're greater than 0
blendDEM_shifted = blendDEM - blendDEM.min() - blendDEM.max()
ctxDEM_shifted = ctxDEM - ctxDEM.min() - ctxDEM.max()

# Take the log10 of the shifted DEMs
blendDEM_log = np.log10(blendDEM_shifted)
ctxDEM_log = np.log10(ctxDEM_shifted)

clipped_imgs = [blendDEM_log, blendSLOPE, ctxDEM_log, ctxIMG, ctxSLOPE_trim, dayIR, nightIR, hrscND, hrscP1, hrscP2, hrscS1, hrscS2]
normalized_imgs = [normalize_image(img) for img in clipped_imgs]

# for img, name in zip(normalized_imgs, clipped_names):
#     plot_image_and_histogram(img, name)

### Match the resolution

In [39]:
for img, name in zip(normalized_imgs, clipped_names):
    length, width = img.shape[1], img.shape[2]
    print(f"Size of {name}: {length} x {width}, Total size: {img.size}")

Size of blendDEM: 72 x 76, Total size: 5472
Size of blendSLOPE: 72 x 76, Total size: 5472
Size of ctxDEM: 798 x 775, Total size: 618450
Size of ctxIMG: 2873 x 3048, Total size: 8756904
Size of ctxSLOPE: 798 x 775, Total size: 618450
Size of dayIR: 144 x 152, Total size: 21888
Size of nightIR: 144 x 152, Total size: 21888
Size of hrscND: 1149 x 1118, Total size: 1284582
Size of hrscP1: 574 x 559, Total size: 320866
Size of hrscP2: 574 x 559, Total size: 320866
Size of hrscS1: 1149 x 1118, Total size: 1284582
Size of hrscS2: 1149 x 1118, Total size: 1284582


In [40]:
# Find the largest image dimensions
max_length = max(img.shape[1] for img in normalized_imgs)
max_width = max(img.shape[2] for img in normalized_imgs)

# Define the target shape
target_shape = (max_length, max_width)

# Function to resize an image
def resize_image(image, target_shape):
    return image.rio.reproject(
        dst_crs=image.rio.crs,
        shape=target_shape,
        resampling=rio.enums.Resampling.bilinear
    )

# Resize all images
resized_imgs = [resize_image(img, target_shape) for img in normalized_imgs]

# Verify the sizes
for img, name in zip(resized_imgs, clipped_names):
    length, width = img.shape[1], img.shape[2]
    print(f"Resized size of {name}: {length} x {width}, Total size: {img.size}")

Resized size of blendDEM: 2873 x 3048, Total size: 8756904
Resized size of blendSLOPE: 2873 x 3048, Total size: 8756904
Resized size of ctxDEM: 2873 x 3048, Total size: 8756904
Resized size of ctxIMG: 2873 x 3048, Total size: 8756904
Resized size of ctxSLOPE: 2873 x 3048, Total size: 8756904
Resized size of dayIR: 2873 x 3048, Total size: 8756904
Resized size of nightIR: 2873 x 3048, Total size: 8756904
Resized size of hrscND: 2873 x 3048, Total size: 8756904
Resized size of hrscP1: 2873 x 3048, Total size: 8756904
Resized size of hrscP2: 2873 x 3048, Total size: 8756904
Resized size of hrscS1: 2873 x 3048, Total size: 8756904
Resized size of hrscS2: 2873 x 3048, Total size: 8756904


In [41]:
blendDEM_aiready, blendSLOPE_aiready, ctxDEM_aiready, ctxIMG_aiready, ctxSLOPE_aiready, dayIR_aiready, nightIR_aiready, hrscND_aiready, hrscP1_aiready, hrscP2_aiready, hrscS1_aiready, hrscS2_aiready = resized_imgs

blendDEM_aiready.rio.to_raster(save_path + "blendDEM_aiready.tif")
blendSLOPE_aiready.rio.to_raster(save_path + "blendSLOPE_aiready.tif")
ctxDEM_aiready.rio.to_raster(save_path + "ctxDEM_aiready.tif")
ctxIMG_aiready.rio.to_raster(save_path + "ctxIMG_aiready.tif")
ctxSLOPE_aiready.rio.to_raster(save_path + "ctxSLOPE_aiready.tif")
dayIR_aiready.rio.to_raster(save_path + "dayIR_aiready.tif")
nightIR_aiready.rio.to_raster(save_path + "nightIR_aiready.tif")
hrscND_aiready.rio.to_raster(save_path + "hrscND_aiready.tif")
hrscP1_aiready.rio.to_raster(save_path + "hrscP1_aiready.tif")
hrscP2_aiready.rio.to_raster(save_path + "hrscP2_aiready.tif")
hrscS1_aiready.rio.to_raster(save_path + "hrscS1_aiready.tif")
hrscS2_aiready.rio.to_raster(save_path + "hrscS2_aiready.tif")

/Users/tthomas/mambaforge/envs/saha_seg/lib/python3.11/site-packages/rioxarray/raster_writer.py:130: UserWarning: The nodata value (3.402823466e+38) has been automatically changed to (3.4028234663852886e+38) to match the dtype of the data.
  warnings.warn(


## Plot the ai-ready images for confirmation

In [34]:
# for img, name in zip(resized_imgs, clipped_names):
#     plot_image_and_histogram(img, name)

In [ ]:
# Marine suggests using Standard Scaling Normalization to normalize the data
# This will scale the data to have a mean of 0 and a standard deviation of 1, making the distributions more comparable